In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/PhD/CMPBP`


In [2]:
using LinearAlgebra, Statistics, Revise
includet("../src/observables.jl")
includet("../src/truncation.jl")
includet("../src/glauber.jl")
includet("../src/cmpbp.jl")

## Lagrange ascent

In [3]:
function build_S(A,B; q=size(A, 1), da=size(A,4), db=size(B,4))
    sum(kron(E(z,zz;q), I(q), I(da), I(db)) for z in 1:q, zz in 1:q if z≠zz) + 
        sum(kron(E(z,z;q), E(x,x;q), kron(A[x,x,z,:,:],I(db))+kron(I(da),B[x,x,z,:,:])) for z in 1:q, x in 1:q) +
        sum(kron(E(z,z;q), E(x,y;q), A[x,y,z,:,:], B[x,y,z,:,:]) for z in 1:q, x in 1:q, y in 1:q if y≠x)
end

function eigmax_S(A,B)
    q = size(A, 1)
    da, db = size(A,4), size(B,4)
    size(A,2)==size(A,3)==size(B,1)==size(B,2)==size(B,3)==q || error("Incompatible dimensions")
    (size(A,5)==da && size(B,5)==db) || error("Incompatible dimensions")

    S = build_S(A,B; q,da,db)
    maximum(real, eigen(S).values)
    # eigen(S)
end

fidelity_eig(A,B) = eigmax_S(A,B) - 0.5 * (eigmax_S(A,A) + eigmax_S(B,B))

fidelity_eig (generic function with 1 method)

In [4]:
q = 2
da, db = 5, 5

B = rand(q,q,q,da,da)
A = rand(q,q,q,db,db)
# A = copy(B) .+ 0.1.*rand.()
t = Truncator(A,B)

@show marginals(B) marginals(A) fidelity(A,B; maxiter=10^6) fidelity_eig(A,B);

marginals(B) = ComplexF64[0.49384452430278103 - 1.6034038755913386e-17im, 0.5061554756972191 + 1.6034038755913383e-17im]
marginals(A) = ComplexF64[0.4733164175900434 + 2.67993588326536e-17im, 0.5266835824099567 - 2.67993588326536e-17im]
fidelity(A, B; maxiter = 10 ^ 6) = -0.02308615740470188
fidelity_eig(A, B) = -0.023086157404698326


In [5]:
ascent!(A, B, t; maxiters=[1000], ηs=[1e-4])
@show marginals(B) marginals(A) fidelity_eig(A,B);

marginals(B) = ComplexF64[0.49384452430278103 - 1.6034038755913386e-17im, 0.5061554756972191 + 1.6034038755913383e-17im]
marginals(A) = ComplexF64[0.48914154333404286 - 2.1067447896930892e-17im, 0.5108584566659572 + 2.106744789693089e-17im]
fidelity_eig(A, B) = -0.0020299305267705847


## Augmented Lagrangian method

In [6]:
e(x; q=2) = [i==x for i in 1:q]

function aug_lagrangian(A, B, Q, P, Q1, P1, λ, λ1; μ=1e2)
    q = size(A, 1)
    da, db = size(A,4), size(B,4)

    S = build_S(A,B; q,da,db)
    S1 = build_S(A,A; q,da,db=da)
    vecQ = sum(kron(e(z;q), e(x;q), vec(Q[x,z,:,:])) for z in 1:q, x in 1:q)
    vecQ1 = sum(kron(e(z;q), e(x;q), vec(Q1[x,z,:,:])) for z in 1:q, x in 1:q)
    vecP = sum(kron(e(z;q), e(x;q), vec(P[x,z,:,:])) for z in 1:q, x in 1:q)
    vecP1 = sum(kron(e(z;q), e(x;q), vec(P1[x,z,:,:])) for z in 1:q, x in 1:q)

    Sq = S*vecQ - λ*vecQ
    S1q1 = S1*vecQ1 - λ1*vecQ1

    return λ - λ1/2 + μ/2 * (Sq'Sq + S1q1'S1q1) + vecP'Sq + vecP1'S1q1
end

aug_lagrangian (generic function with 1 method)

In [83]:
q = 2
da, db = 5, 5

A = rand(q,q,q,da,da)
# B = rand(q,q,q,db,db)
B = copy(A) #.+ 1e-2.*rand.()
@show fidelity_eig(A,B)

t = Truncator(A,B);

fidelity_eig(A, B) = 0.0


In [84]:
Q, P, Q1, P1 = t.Q, t.P, t.Q1, t.P1
Qold, Pold, Q1old, P1old = t.Qold, t.Pold, t.Q1old, t.P1old
simQ, simQ1, X = t.simQ, t.simQ1, t.X

maxiter_pow = 10^5
tol_pow = 1e-20

findeigen_l!(P, A, B; maxiter_pow, tol_pow, q,da,db, X=simQ, Pold)
findeigen_r!(Q, A, B; maxiter_pow, tol_pow, q,da,db, X=simQ, Qold)
findeigen_l!(P1, A, A; maxiter_pow, tol_pow, q,da,db=da, X=simQ1, Pold=P1old)
findeigen_r!(Q1, A, A; maxiter_pow, tol_pow, q,da,db=da, X=simQ1, Qold=Q1old)
n = dotprod(P, Q)
P ./= n
n1 = dotprod(P1, Q1)
P1 ./= -2n1

Qold .= Q
Pold .= P
Q1old .= Q1
P1old .= P1

apply!(Qold,A,B)
apply_dag!(Pold,A,B)
apply!(Q1old,A,A)
apply_dag!(P1old,A,A)

t.λ = mean(Qold./Q)
t.λ1 = mean(Q1old./Q1)
Qold .= Q
Q1old .= Q1
;

maxiter_pow = 100000
maxiter_pow = 100000
maxiter_pow = 100000
maxiter_pow = 100000


In [85]:
# A = rand(q,q,q,da,da)
# B = rand(q,q,q,db,db)
# # B = copy(A) .+ 1e-2.*rand.()

# t = Truncator(A,B)

In [88]:
using ForwardDiff

autoderA = ForwardDiff.gradient(A->aug_lagrangian(A,B,t.Q,t.P,t.Q1,t.P1,t.λ,t.λ1), A)
autoderQ = ForwardDiff.gradient(Q->aug_lagrangian(A,B,Q,t.P,t.Q1,t.P1,t.λ,t.λ1), t.Q)
autoderP = ForwardDiff.gradient(P->aug_lagrangian(A,B,t.Q,P,t.Q1,t.P1,t.λ,t.λ1), t.P)
autoderQ1 = ForwardDiff.gradient(Q1->aug_lagrangian(A,B,t.Q,t.P,Q1,t.P1,t.λ,t.λ1), t.Q1)
autoderP1 = ForwardDiff.gradient(P1->aug_lagrangian(A,B,t.Q,t.P,t.Q1,P1,t.λ,t.λ1), t.P1)
autoderλ = ForwardDiff.derivative(λ->aug_lagrangian(A,B,t.Q,t.P,t.Q1,t.P1,λ,t.λ1), t.λ)
autoderλ1 = ForwardDiff.derivative(λ1->aug_lagrangian(A,B,t.Q,t.P,t.Q1,t.P1,t.λ,λ1), t.λ1)
;

In [89]:
derλ, derλ1, derQ, derP, derQ1, derP1, derA = aug_lagrange!(A,B, t; μs=[1e2], maxiters=[1], ηs=[1e-1], tols=[1e-12]);

∂/∂λ
2.0540504079810816e-13
∂/∂λ1
1.6832166629787787e-13
∇Q
6.875568799353307e-11
∇P
6.428191312579656e-14
∇Q1
4.3190780766625653e-11
∇P1
4.7128967395337895e-14
∇A
8.726455921363612e-13

marginals(B) = ComplexF64[0.4898750397064748 - 1.5966858346700673e-17im, 0.5101249602935252 + 1.5966858346700673e-17im]
marginals(A) = ComplexF64[0.48987503970648083 - 1.2563254829870378e-17im, 0.5101249602935192 + 1.2563254829870375e-17im]
fidelity_eig(A,B) = 2.842170943040401e-14


In [90]:
@show maximum(abs, (derA .- autoderA) ./ autoderA)
@show maximum(abs, (derQ .- autoderQ))
@show maximum(abs, (derP .- autoderP))
@show maximum(abs, (derQ1 .- autoderQ1))
@show maximum(abs, (derP1 .- autoderP1))
@show maximum(abs, (derλ .- autoderλ))
@show maximum(abs, (derλ1 .- autoderλ1))
;

maximum(abs, (derA .- autoderA) ./ autoderA) = 0.42308209240546585
maximum(abs, derQ .- autoderQ) = 6.793024800994646e-14
maximum(abs, derP .- autoderP) = 5.551115123125783e-17
maximum(abs, derQ1 .- autoderQ1) = 6.933510584548524e-14
maximum(abs, derP1 .- autoderP1) = 5.551115123125783e-17
maximum(abs, derλ .- autoderλ) = 1.3781242454202061e-17
maximum(abs, derλ1 .- autoderλ1) = 6.5427805007096e-16


In [92]:
(derA .- autoderA) ./ autoderA

2×2×2×5×5 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 0.00428783   0.00330612
 0.000582364  0.000637387

[:, :, 2, 1, 1] =
 -0.00433818  -0.000665725
  0.00116178   0.00112565

[:, :, 1, 2, 1] =
 -9.64665e-5  -0.00010027
  0.0023665    0.00349471

[:, :, 2, 2, 1] =
 0.00254498   0.0033817
 0.000207389  0.000295473

[:, :, 1, 3, 1] =
 -0.00128106  -0.00128074
 -0.0163645   -0.0511933

[:, :, 2, 3, 1] =
 -0.00120244  -0.00105976
  0.0157324    2.75232e-5

[:, :, 1, 4, 1] =
 0.00134028  0.00163237
 0.00535454  0.00412392

[:, :, 2, 4, 1] =
  0.00255515   0.00262159
 -0.000135793  0.000190916

[:, :, 1, 5, 1] =
  0.000167619  0.000145741
 -0.0164358    0.0473881

[:, :, 2, 5, 1] =
 -0.0698958    0.13365
 -0.00171651  -0.0021175

[:, :, 1, 1, 2] =
 0.00416618   0.00346109
 0.000698925  0.00051263

[:, :, 2, 1, 2] =
 -0.00422151  -0.000940963
  0.00105911   0.00107524

[:, :, 1, 2, 2] =
 -9.18906e-5  -0.00011511
  0.00266457   0.00286381

[:, :, 2, 2, 2] =
 0.00296956   0.00259668
 0.000230703  0.